# client

> Client for interacting with the Fewsats API

In [1]:
#| default_exp core

In [2]:
#| export
from fastcore.utils import *
import os
import httpx
from typing import Dict, Any, List
import json
from fastcore.basics import BasicRepr
from fastcore.utils import store_attr
from typing import List, Dict, Any

In [3]:
#| hide 
from dotenv import load_dotenv
from fastcore.test import *

In [4]:
#| hide
load_dotenv()

True

The `Fewsats` class handles authentication and provides the foundation for our API interactions.

In [5]:
#| export
class Fewsats:
    "Client for interacting with the Fewsats API"
    def __init__(self,
                 api_key: str = None, # The API key for the Fewsats account
                 base_url: str = "https://api.fewsats.com"): # The Fewsats API base URL
        self.api_key = api_key or os.environ.get("FEWSATS_API_KEY")
        if not self.api_key:
            raise ValueError("The api_key client option must be set either by passing api_key to the client or by setting the FEWSATS_API_KEY environment variable")
        self.base_url = base_url
        self._httpx_client = httpx.Client()
        self._httpx_client.headers.update({"Authorization": f"Token {self.api_key}"})


In [6]:
k = os.getenv("FEWSATS_API_KEY")
fs = Fewsats(api_key=k)
# k = os.getenv("FEWSATS_LOCAL_API_KEY")
# fs = Fewsats(api_key=k, base_url="http://localhost:8000")

test_eq(fs.api_key, k)
test_eq(fs._httpx_client.headers["Authorization"], f"Token {k}")

## Methods

In [7]:
#| export
@patch
def _request(self: Fewsats, 
             method: str, # The HTTP method to use
             path: str, # The path to request
             timeout: int = 10, # Timeout for the request in s
             **kwargs) -> Dict[str, Any]:
    "Makes an authenticated request to Fewsats API"
    url = f"{self.base_url}/{path}"
    return  self._httpx_client.request(method, url, timeout=timeout, **kwargs)

In [8]:
r  = fs._request("GET", "v0/users/me")
test_eq(r.status_code, 200)

Let's use a helper function to process the response.

### User Info

In [9]:
#| export

@patch
def me(self: Fewsats):
    "Retrieve the user's info."
    return self._request("GET", "v0/users/me")


In [10]:
r = fs.me()
r.status_code, r.json()

(200,
 {'name': 'Fewsats',
  'last_name': 'Tester',
  'email': 'test@fewsats.com',
  'billing_info': None,
  'id': 15,
  'created_at': '2024-12-18T18:19:00.531Z',
  'webhook_url': 'https://example.com'})

### Balance 

In [11]:
#| export 

@patch
def balance(self: Fewsats):
    "Retrieve the balance of the user's wallet."
    return self._request("GET", "v0/wallets")


In [12]:
r = fs.balance()
r.status_code, r.json()

(200, [{'id': 15, 'balance': 359, 'currency': 'usd'}])

### Payment Methods

Retrieve the user's payment methods. Useful for checking which card will be used for purchases.

In [13]:
#| export
@patch
def payment_methods(self: Fewsats) -> List[Dict[str, Any]]:
    "Retrieve the user's payment methods, raises an exception for error status codes."
    return self._request("GET", "v0/stripe/payment-methods")


In [14]:
r = fs.payment_methods()
payment_methods = r.json()
r.status_code, payment_methods

(200,
 [{'id': 5,
   'last4': '4242',
   'brand': 'Visa',
   'exp_month': 12,
   'exp_year': 2034,
   'is_default': True}])

In [15]:
assert isinstance(payment_methods, list)

### Preview a Purchase

Preview the resulting state of a purchase. Useful, for example, to check if a CC charge is needed or the purchase will use the balance.

In [16]:
#| export

@patch
def _preview_payment(self: Fewsats,
                    amount: str): # The amount in USD cents
    "Simulates a purchase, raises an exception for error status codes."
    assert amount.isdigit()
    return self._request("POST", "v0/l402/preview/purchase/amount", json={"amount_usd": amount})


In [17]:
r = fs._preview_payment(amount="300") # 3.00 USD
preview = r.json()
r.status_code = preview

### Create offers

How to use the client to generate L402 offers

In [18]:
#| export
@patch
def create_offers(self:Fewsats,
                 offers:List[Dict[str,Any]], # List of offer objects following OfferCreateV0 schema
) -> dict:
    "Create offers for L402 payment server"
    return self._request("POST", "v0/l402/offers", json={"offers": offers})

In [19]:
test_offers = [{
    "offer_id": "test_offer_2",
    "amount": 1,
    "currency": "usd" ,
    "description": "Test offer",
    "title": "Test Package",
    "payment_methods": ["lightning", "credit_card"]
}]

r = fs.create_offers(test_offers)
l402_offers = r.json()
r.status_code, l402_offers

(200,
 {'offers': [{'offer_id': 'test_offer_2',
    'amount': 1,
    'currency': 'usd',
    'description': 'Test offer',
    'title': 'Test Package',
    'payment_methods': ['lightning', 'credit_card'],
    'type': 'one-off'}],
  'payment_context_token': 'b40eb56c-5d3f-42ba-8cff-fc5d095299ee',
  'payment_request_url': 'https://api.fewsats.com/v0/l402/payment-request',
  'version': '0.2.2'})

### Get Payment Details

In [20]:
#| export
@patch
def get_payment_details(self:Fewsats,
                       payment_request_url:str,
                       offer_id:str,
                       payment_method:str,
                       payment_context_token:str,
                       ) -> dict:
    data = {"offer_id": offer_id, "payment_method": payment_method, "payment_context_token": payment_context_token}
    return httpx.post(payment_request_url, json=data)


In [21]:
r = fs.get_payment_details(l402_offers["payment_request_url"], l402_offers["offers"][0]["offer_id"], "lightning", l402_offers["payment_context_token"])
payment_details = r.json()
ln_invoice = payment_details["payment_request"]['lightning_invoice']
r.status_code, payment_details

(200,
 {'expires_at': '2025-03-11T06:11:06.060263+00:00',
  'offer_id': 'test_offer_2',
  'payment_request': {'lightning_invoice': 'lnbc120n1pnulj78pp5ezjj3xc0edx4zeegp6grv9a6v45x8d4fc8taqdskssnt05lquyrsdq523jhxapq2pskx6mpvajscqzpgxqrzpjrzjqwghf7zxvfkxq5a6sr65g0gdkv768p83mhsnt0msszapamzx2qvuxqqqqz99gpz55yqqqqqqqqqqqqqq9qrzjq25carzepgd4vqsyn44jrk85ezrpju92xyrk9apw4cdjh6yrwt5jgqqqqz99gpz55yqqqqqqqqqqqqqq9qsp5gvpty2zjk4hpcxdzqugfujsfu7gue76nzhgxhjk3kvuqacqaqf2q9qxpqysgqeu7n5sq6wcuy3xmeju2lnh3lkwgsd7c5me95kntdayg2v30g96l448d8uhge3w59w8fm3987xfl64p74qa8evhjhaz0h4gus79ruz8sp46ggg2'},
  'version': '0.2.2'})

In [22]:
r.is_success

True

### Get Payment Status


In [23]:
#| export
@patch
def get_payment_status(self:Fewsats,
                       payment_context_token:str,
                       ) -> dict:
    return self._request("GET", f"v0/l402/payment-status?payment_context_token={payment_context_token}")

In [24]:

r = fs.get_payment_status(l402_offers["payment_context_token"])
r.status_code, r.json()

(200,
 {'payment_context_token': 'b40eb56c-5d3f-42ba-8cff-fc5d095299ee',
  'status': 'pending',
  'offer_id': None,
  'paid_at': None,
  'amount': None,
  'currency': None})

In [25]:
#| export
@patch
def set_webhook(self:Fewsats,
                       webhook_url:str,
                       ) -> dict:
    return self._request("POST", f"v0/users/webhook/set", json={"webhook_url": webhook_url})

In [26]:

r = fs.set_webhook("https://example.com")
r, r.json()

(<Response [200 OK]>,
 {'name': 'Fewsats',
  'last_name': 'Tester',
  'email': 'test@fewsats.com',
  'billing_info': None,
  'id': 15,
  'created_at': '2024-12-18T18:19:00.531Z',
  'webhook_url': 'https://example.com'})

### Pay Lightning Invoice

In [27]:
#| export

@patch
def pay_lightning(self: Fewsats, 
                  invoice: str, # lightning invoice
                  amount: int, # amount in cents
                  currency: str = "usd", # currency
                  description: str = "" ): # description of the payment 
    "Pay for a lightning invoice"
    data = {
        "invoice": invoice,
        "amount": amount,
        "currency": currency,
        "description": description
    }
    return self._request("POST", "v0/l402/purchases/lightning", json=data)

In [28]:
r = fs.pay_lightning(invoice=ln_invoice,
                     description="fewsats webhook trial", amount=1)
lightning_payment = r.json()
r.status_code, lightning_payment

(200,
 {'id': 412,
  'created_at': '2025-03-11T05:36:09.026Z',
  'status': 'success',
  'payment_request_url': '',
  'payment_context_token': '',
  'invoice': 'lnbc120n1pnulj78pp5ezjj3xc0edx4zeegp6grv9a6v45x8d4fc8taqdskssnt05lquyrsdq523jhxapq2pskx6mpvajscqzpgxqrzpjrzjqwghf7zxvfkxq5a6sr65g0gdkv768p83mhsnt0msszapamzx2qvuxqqqqz99gpz55yqqqqqqqqqqqqqq9qrzjq25carzepgd4vqsyn44jrk85ezrpju92xyrk9apw4cdjh6yrwt5jgqqqqz99gpz55yqqqqqqqqqqqqqq9qsp5gvpty2zjk4hpcxdzqugfujsfu7gue76nzhgxhjk3kvuqacqaqf2q9qxpqysgqeu7n5sq6wcuy3xmeju2lnh3lkwgsd7c5me95kntdayg2v30g96l448d8uhge3w59w8fm3987xfl64p74qa8evhjhaz0h4gus79ruz8sp46ggg2',
  'preimage': 'a7d502c0fccc9c66116d80ca2ff760cbcedd1269eb64eca298b856aefe6783c6',
  'amount': 1,
  'currency': 'usd',
  'payment_method': 'lightning',
  'title': '',
  'description': 'fewsats webhook trial',
  'type': '',
  'is_test': False})

In [29]:
fs.get_payment_status(l402_offers["payment_context_token"]).json()


{'payment_context_token': 'b40eb56c-5d3f-42ba-8cff-fc5d095299ee',
 'status': 'pending',
 'offer_id': None,
 'paid_at': None,
 'amount': None,
 'currency': None}

In [30]:
fs.get_payment_status('8d87cec8-5d5d-4f1f-a82a-223fe81a5394').json()

{'payment_context_token': '8d87cec8-5d5d-4f1f-a82a-223fe81a5394',
 'status': 'success',
 'offer_id': 'simple-offer-1741619322607',
 'paid_at': '2025-03-10T15:08:54.268250+00:00',
 'amount': 1,
 'currency': 'USD'}

### Pay Offer

The pay method pays for a specific offer. The user is not required to fetch the payment details beforehand. It is asynchronous and returns the `payment_id` and `status`. Using the `payment_id` we can check the status of the payment.


There are two versions of the pay offer. One accepts the L402 offers as a string, which is more convenient for text-based AI agents like LLMs. The other uses custom L402 offer classes, which are supported by some libraries like Claudette](https://claudette.answer.ai/core.html#tool)

In [31]:
#| export

class Offer(BasicRepr):
    "Represents a single L402 offer"
    def __init__(self, 
                 offer_id: str,
                 amount: int,
                 currency: str,
                 description: str,
                 title: str,
                 payment_methods: List[str] = None,
                 type: str = "one-off"): 
        store_attr()

    def __repr__(self):
        return f"Offer: {self.title}\nID: {self.offer_id}\nAmount: {self.amount/100} {self.currency}\nDescription: {self.description}"
    
    @classmethod
    def from_dict(cls, d: Dict[str, Any]) -> 'Offer':
        "Create an Offer from a dictionary"
        return cls(**d)

class L402Offers(BasicRepr):
    "Represents the complete L402 offers schema"
    def __init__(self, 
                 offers: List[Offer],
                 payment_context_token: str,
                 payment_request_url: str,
                 version: str): 
        store_attr()
    
    def __repr__(self):
        offers_str = "\n".join([f"- {o.title} ({o.amount/100} {o.currency})" for o in self.offers])
        return f"L402 Offers:\n{offers_str}\nPayment URL: {self.payment_request_url}\nContext Token: {self.payment_context_token}"
    
    def as_dict(self) -> Dict[str, Any]:
        "Convert to dictionary format for API usage"
        return {
            'offers': [vars(o) for o in self.offers],
            'payment_context_token': self.payment_context_token,
            'payment_request_url': self.payment_request_url,
            'version': self.version
        }
    
    @classmethod
    def from_dict(cls, d: Dict[str, Any]) -> 'L402Offers':
        "Create an L402Offers object from a dictionary"
        offers = [Offer.from_dict(o) for o in d['offers']]
        return cls(
            offers=offers,
            payment_context_token=d['payment_context_token'],
            payment_request_url=d['payment_request_url'],
            version=d['version']
        )


In [32]:
l402 = L402Offers.from_dict(l402_offers)
l402

L402 Offers:
- Test Package (0.01 usd)
Payment URL: https://api.fewsats.com/v0/l402/payment-request
Context Token: b40eb56c-5d3f-42ba-8cff-fc5d095299ee

In [33]:
# we make sure that invalid offers fail
invalid_json = {
    'offers': [{'offer_id': 'test_offer_2', 'amount': 1}],  # Missing fields
    'payment_context_token': '60a8e027-8b8b-4ccf-b2b9-380ed0930283'
    # Missing payment_request_url
}
test_fail(lambda: Offer(offer_id="test", amount=1), 
          contains="missing 3 required positional arguments: 'currency', 'description', and 'title'")


### Pay Offer with L402Offers object


In [34]:
#| export 

@patch
def pay_offer(self:Fewsats,
        offer_id : str, # the offer id to pay for
        l402_offer: L402Offers, # a dictionary containing L402 offers
) -> dict: # payment status response
    """Pays an offer_id from the l402_offers. This tools requires the LLM caller to 
    support custom classes as parameters like Claudette does.

    Returns payment status response"""
    if isinstance(l402_offer, dict): l402_offer = L402Offers.from_dict(l402_offer)
    offer_dict = l402_offer.as_dict()
    data = {"offer_id": offer_id, **offer_dict}
    return self._request("POST", "v0/l402/purchases/from-offer", json=data)


In [35]:
offer_id = l402.offers[0].offer_id
result = fs.pay_offer(offer_id, l402)

### Pay Offer with JSON string

This alternative method accepts a JSON string containing L402 offers. It should be used by systems that do not support custom classes in tool calling.

In [36]:
#| export

@patch
def pay_offer_str(self:Fewsats,
        offer_id : str, # the offer id to pay for
        l402_offer: str, # JSON string containing L402 offers
) -> dict: # payment status response
    """Pays an offer_id from the l402_offers.

    The l402_offer parameter must be a JSON string with this structure:
    {
        'offers': [
            {
                'offer_id': 'test_offer_2',  # String identifier for the offer
                'amount': 1,                 # Numeric cost value
                'currency': 'usd',           # Currency code
                'description': 'Test offer', # Text description
                'title': 'Test Package'      # Title of the package
            }
        ],
        'payment_context_token': '60a8e027-8b8b-4ccf-b2b9-380ed0930283',  # Payment context token
        'payment_request_url': 'https://api.fewsats.com/v0/l402/payment-request',  # Payment URL
        'version': '0.2.2'  # API version
    }

    Returns payment status response"""
    # Parse JSON string to dictionary
    try:
        offer_data = json.loads(l402_offer)
        L402Offers.from_dict(offer_data) # we don't care about the return value, just validating the json input
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON string provided for l402_offer")
    
    # Create payload with offer_id
    data = {"offer_id": offer_id, **offer_data}
    
    return self._request("POST", "v0/l402/purchases/from-offer", timeout=20, json=data)

In [37]:
r = fs.pay_offer_str(l402_offers["offers"][0]["offer_id"], json.dumps(l402_offers))
payment_response = r.json() if r.is_success else r.text
r.status_code, payment_response

(200,
 {'id': 414,
  'created_at': '2025-03-11T05:36:19.100Z',
  'status': 'success',
  'payment_method': 'lightning'})

In [39]:
json.dumps(l402_offers)

'{"offers": [{"offer_id": "test_offer_2", "amount": 1, "currency": "usd", "description": "Test offer", "title": "Test Package", "payment_methods": ["lightning", "credit_card"], "type": "one-off"}], "payment_context_token": "b40eb56c-5d3f-42ba-8cff-fc5d095299ee", "payment_request_url": "https://api.fewsats.com/v0/l402/payment-request", "version": "0.2.2"}'

In [ ]:
{
"offers": [
    {
    "type": "one-off",
    "title": "Test Purchase",
    "amount": 100,
    "currency": "USD",
    "offer_id": "x",
    "description": "Small test purchase for Agora",
    "payment_methods": [
        "lightning",
        "credit_card"
    ]
    }
],
"version": "0.2.2",
"payment_request_url": "https://api.fewsats.com/v0/l402/payment-request",
"payment_context_token": "cf25f915-dd4d-4c42-81e5-74b2960edd03"
}

(400,
 '{"detail": "Invalid payment request received. Could not get payment details. {\\"detail\\": \\"Invalid payment request received. Payment context token \'b40eb56c-5d3f-42ba-8cff-fc5d095299ee\' already used\\"}"}')

In [ ]:
l402_offers_json = {
  "offer_id": "test_purchase_claude_2025",
  "l402_offers": {
    "offers": [
      {
        "type": "one-off",
        "title": "Test Purchase",
        "amount": 100,
        "currency": "USD",
        "offer_id": "test_purchase_claude_2025",
        "description": "Small test purchase for Agora",
        "payment_methods": [
          "lightning",
          "credit_card"
        ]
      }
    ],
    "version": "0.2.2",
    "payment_request_url": "https://api.fewsats.com/v0/l402/payment-request",
    "payment_context_token": "cf25f915-dd4d-4c42-81e5-74b2960edd03"
  }
}

r = fs.pay_offer_str(l402_offers["offers"][0]["offer_id"], json.dumps(l402_offers))
payment_response = r.json() if r.is_success else r.text
r.status_code, payment_response

After the stripe payment settles, the status will be updated to `success`.

### Payment Info

We can check the status of a payment as follows:

In [ ]:
#| export
@patch
def payment_info(self:Fewsats,
                  pid:str): # purchase id
    "Retrieve the details of a payment."
    return self._request("GET", f"v0/l402/outgoing-payments/{pid}")

In [ ]:
r = fs.payment_info(payment_response['id'])
r.status_code, r.json()

(200,
 {'id': 161,
  'created_at': '2025-03-10T14:17:37.830Z',
  'status': 'success',
  'payment_request_url': 'http://localhost:8000/v0/l402/payment-request',
  'payment_context_token': '8a0060db-cb67-4dd3-9fe3-8ee9a2d37d9c',
  'invoice': 'lnbc120n1pnuaayppp5xg8smav2tc4r6gg3tkavsxweq5zg7gqn0kh86xjedjg6zcttlmusdq523jhxapq2pskx6mpvajscqzpgxqrzpnrzjqwghf7zxvfkxq5a6sr65g0gdkv768p83mhsnt0msszapamzx2qvuxqqqqz99gpz55yqqqqqqqqqqqqqq9qrzjq25carzepgd4vqsyn44jrk85ezrpju92xyrk9apw4cdjh6yrwt5jgqqqqz99gpz55yqqqqqqqqqqqqqq9qsp5rs09430wvr6uhpw2yay33wur05nfs70vcsyf8uskc7cmksvvg8sq9qxpqysgqguwdpjc0tqlz2umcyjc07pjcg8ag4ads483mgzn0ahe5evulfrw4nc3kt0dwk7e46v286unkg0xm4qesxkjm6wk0392fl9hexs7mf3cpv4c6p2',
  'preimage': 'b019dd7257a0301164d9cd5dccb0e9de8665fbec0be675948d41473d7509c187',
  'amount': 1,
  'currency': 'usd',
  'payment_method': 'lightning',
  'title': 'Test Package',
  'description': 'Test offer',
  'type': 'one-off',
  'is_test': False})

## As tools

In [ ]:
#| export

@patch
def as_tools(self:Fewsats):
    "Return list of available tools for AI agents"
    return [
        self.me,
        self.balance,
        self.payment_methods,
        self.pay_offer_str,
        self.payment_info,
    ]

In [ ]:
fs.as_tools()

[<bound method Fewsats.me of <__main__.Fewsats object>>,
 <bound method Fewsats.balance of <__main__.Fewsats object>>,
 <bound method Fewsats.payment_methods of <__main__.Fewsats object>>,
 <bound method Fewsats.pay_offer_str of <__main__.Fewsats object>>,
 <bound method Fewsats.payment_info of <__main__.Fewsats object>>]

Both the preview and purchase methods automatically use the default payment method if a charge is needed. This client provides a straightforward way to interact with the Fewsats API, making it easy for developers to integrate Fewsats functionality into their applications.

## Agent Demo

We will use [Claudette](https://claudette.answer.ai/) to demonstrate how to pay for content using the Fewsats API.

In [ ]:
from claudette import Chat, models

In [ ]:
model = models[1]
model

'claude-3-5-sonnet-20240620'

In [ ]:
fs.balance()

<Response [200 OK]>

In [ ]:
chat = Chat(model, sp='You are a helpful assistant that can pay offers.', tools=fs.as_tools())
pr = f"Could you pay the cheapest offer in {l402_offers}?"
r = chat.toolloop(pr, trace_func=print)
r

Message(id='msg_01RWLkNPGHMsEaFQ6TqTm2dV', content=[TextBlock(text="Certainly! I'll analyze the offer information you provided and pay for the cheapest offer. In this case, there's only one offer available, so that will be the one we'll proceed with.\n\nLet's use the `pay_offer_str` function to make the payment. Here's what we'll do:\n\n1. Prepare the L402 offer string.\n2. Identify the offer ID (which is 'test_offer_2' in this case).\n3. Call the `pay_offer_str` function with the required parameters.\n\nHere's the function call:", type='text'), ToolUseBlock(id='toolu_01SoFW381vm54dYBY8wx7Lye', input={'offer_id': 'test_offer_2', 'l402_offer': '{"offers": [{"offer_id": "test_offer_2", "amount": 1, "currency": "usd", "description": "Test offer", "title": "Test Package", "payment_methods": ["lightning", "credit_card"], "type": "one-off"}], "payment_context_token": "8a0060db-cb67-4dd3-9fe3-8ee9a2d37d9c", "payment_request_url": "http://localhost:8000/v0/l402/payment-request", "version": "0.

Great! The payment has been processed successfully. The response code 200 OK indicates that the transaction was completed without any issues.

To summarize:
- We paid for the offer with ID 'test_offer_2'.
- The cost was 1 USD.
- The offer was titled "Test Package" with the description "Test offer".
- The payment was successful.

Is there anything else you would like to know about this transaction or any other assistance you need?

<details>

- id: `msg_01V2BitQuF4qmLho6cRBJCBi`
- content: `[{'text': 'Great! The payment has been processed successfully. The response code 200 OK indicates that the transaction was completed without any issues.\n\nTo summarize:\n- We paid for the offer with ID \'test_offer_2\'.\n- The cost was 1 USD.\n- The offer was titled "Test Package" with the description "Test offer".\n- The payment was successful.\n\nIs there anything else you would like to know about this transaction or any other assistance you need?', 'type': 'text'}]`
- model: `claude-3-5-sonnet-20240620`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1393, 'output_tokens': 106}`

</details>

We can see in the chat history to see that the agent correctlye filled the required information for the payment.

The payment balance has also decreased as expected.

In [ ]:
fs.balance(), chat.h

(<Response [200 OK]>,
 [{'role': 'user',
   'content': [{'type': 'text',
     'text': "Could you pay the cheapest offer in {'offers': [{'offer_id': 'test_offer_2', 'amount': 1, 'currency': 'usd', 'description': 'Test offer', 'title': 'Test Package', 'payment_methods': ['lightning', 'credit_card'], 'type': 'one-off'}], 'payment_context_token': '8a0060db-cb67-4dd3-9fe3-8ee9a2d37d9c', 'payment_request_url': 'http://localhost:8000/v0/l402/payment-request', 'version': '0.2.2'}?"}]},
  {'role': 'assistant',
   'content': [TextBlock(text="Certainly! I'll analyze the offer information you provided and pay for the cheapest offer. In this case, there's only one offer available, so that will be the one we'll proceed with.\n\nLet's use the `pay_offer_str` function to make the payment. Here's what we'll do:\n\n1. Prepare the L402 offer string.\n2. Identify the offer ID (which is 'test_offer_2' in this case).\n3. Call the `pay_offer_str` function with the required parameters.\n\nHere's the function 

In [ ]:
#|hide
from nbdev.doclinks import nbdev_export
nbdev_export()